In [ ]:
import scipy.io
import numpy as np
import matplotlib.image as img
import matplotlib.pyplot as plt

def readFile():
    M = np.load('./nips-2017-adversarial-learning-development-set/imageNumpy.npy')

    M=M/255.0
    # numImages = M.shape[0]
    # M = np.array([M[i].flatten() for i in range(0,numImages)])
    # print(M[0,:299,:299,0].flatten())

    newM = np.zeros((1000, 3*299*299))
    for i in range(M.shape[0]):
        R = M[i,:299,:299,0].flatten()
        G = M[i,:299,:299,1].flatten()
        B = M[i,:299,:299,2].flatten()

        newM[i,:299*299*1] = R
        newM[i,299*299*1:299*299*2] = G
        newM[i,299*299*2:299*299*3] = B

    print(newM.shape)
    return newM

In [ ]:
def draw_weights(synapses, Kx, Ky):
    yy=0
#     print(synapses[yy,:].shape)
#     A = synapses[yy,:].reshape(height,width, color)
    
    HM=np.zeros((height*Ky,width*Kx, color))
    for y in range(Ky):
        for x in range(Kx):
            
            R = synapses[yy,0:height*width*1] # 299*299
            min_R = np.amin(R)
            if min_R < 0:
                R += abs(min_R)
                
            max_R = np.amax(R)
            R /= max_R
            R = R.reshape(height,width)
            
            G = synapses[yy,height*width*1:height*width*2]
            
            min_G = np.amin(G)
            if min_G < 0:
                G += abs(min_G)
                
            max_G = np.amax(G)
            G /= max_G
            G = G.reshape(height,width)
            
            B = synapses[yy,height*width*2:height*width*3]
            
            min_B = np.amin(B)
            if min_B < 0:
                B += abs(min_B)
                
            max_B = np.amax(B)
            B /= max_B
            B = B.reshape(height,width)
            
            HM[y*height:(y+1)*height,x*width:(x+1)*width,0]=R
            HM[y*height:(y+1)*height,x*width:(x+1)*width,1]=G
            HM[y*height:(y+1)*height,x*width:(x+1)*width,2]=B
            
            
            #print(HM[y*height:(y+1)*height,x*width:(x+1)*width, :])
            yy += 1
    
    plt.clf()
    
#     nc=np.amax(np.absolute(HM))
    im=plt.imshow(HM)
    #fig.colorbar(im,ticks=[np.amin(HM), 0, np.amax(HM)])
    plt.axis('off')
    plt.show()
    fig.canvas.draw()

In [ ]:
M = readFile()

In [ ]:
eps0=2e-2    # learning rate
Kx=10
Ky=10
hid=Kx*Ky    # number of hidden units that are displayed in Ky by Kx array
mu=0.0
sigma=1.0
Nep=2000      # number of epochs
Num=100    # size of the minibatch
prec=1e-30
delta=0.4    # Strength of the anti-hebbian learning
p=3.0        # Lebesgue norm of the weights
k=2

In [ ]:
width = 299
height = 299
color = 3
Nc=10
Ns=1000
N = width*height*color

In [ ]:
%matplotlib inline
%matplotlib notebook
fig=plt.figure(figsize=(10,10))

synapses = np.random.normal(mu, sigma, (hid, N))
draw_weights(synapses, Kx, Ky)
for nep in range(Nep):
    print(nep)
    eps=eps0*(1-nep/Nep)
    M=M[np.random.permutation(Ns),:]
    for i in range(Ns//Num):
        # Num개 씩 잘라서 학습 - batch
        inputs=np.transpose(M[i*Num:(i+1)*Num,:])
        sig=np.sign(synapses)
        tot_input=np.dot(sig*np.absolute(synapses)**(p-1),inputs)

        y=np.argsort(tot_input,axis=0)
        yl=np.zeros((hid,Num))
        yl[y[hid-1,:],np.arange(Num)]=1.0
        yl[y[hid-k],np.arange(Num)]=-delta

        xx=np.sum(np.multiply(yl,tot_input),1)
        ds=np.dot(yl,np.transpose(inputs)) - np.multiply(np.tile(xx.reshape(xx.shape[0],1),(1,N)),synapses)

        nc=np.amax(np.absolute(ds))
        if nc<prec:
            nc=prec

        synapses += eps*np.true_divide(ds,nc)
        
    draw_weights(synapses, Kx, Ky)
    print(synapses.shape)
    print(nep)
